In [27]:
import sys
sys.path.append('/home/miketheologitis/MapReduce-Implementation')

from mapreduce.cluster.local_cluster import LocalCluster

In [28]:
local_cluster = LocalCluster(
    n_workers=0,
    n_masters=10,
    initialize=True,
    verbose=False
)

In [29]:
local_monitoring = local_cluster.local_monitoring

In [30]:
import random
import string

def generate_random_string(str_len):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for _ in range(str_len))

def generate_random_string_list(n, str_len):
    return [generate_random_string(str_len) for _ in range(n)]

# Example usage
#data = generate_random_string_list(n=500000, str_len=20)
data = generate_random_string_list(n=100, str_len=20)

In [31]:
data[:10]

['nnfuaiyfraefaaviqjuq',
 'kvhbrmyxvshfyxpfkmqv',
 'mkawmgedxbuyepmhieiy',
 'guzzcjqhiprwgloqlhcv',
 'qlyzqnovroalemypgage',
 'csklttjrujbzaufyssby',
 'ymquyynsvtciczjotybw',
 'sjcigolsfebpqncrrkfl',
 'shujmzruzxshqtxtdkfi',
 'lcmtorxygwmyizfzcatv']

In [32]:
import sys
print(f"Size: {round(sys.getsizeof(data) / (1024 * 1024), 3)} MB")

Size: 0.001 MB


In [33]:
def map_func(data):
    result = []
    for string in data:
        for char in string:
            result.append((char, 1))
    return result

def reduce_func(data):
    return sum(data)

In [9]:
future1 = local_cluster.mapreduce(data, map_func, reduce_func, requested_n_workers=2)

In [11]:
future1

<Future at 0x7f7fe443bd50 state=running>

In [15]:
future1.result()

[('a', 384993),
 ('b', 384058),
 ('c', 385432),
 ('d', 384097),
 ('e', 384582),
 ('f', 384730),
 ('g', 383708),
 ('h', 385596),
 ('i', 383945),
 ('j', 384994),
 ('k', 384845),
 ('l', 384452),
 ('m', 384036),
 ('n', 383996),
 ('o', 385117),
 ('p', 385352),
 ('q', 385935),
 ('r', 384729),
 ('s', 384508),
 ('t', 384910),
 ('u', 384548),
 ('v', 384364),
 ('w', 384810),
 ('x', 383658),
 ('y', 385116),
 ('z', 383489)]

In [10]:
future2 = local_cluster.mapreduce(data, map_func, reduce_func, requested_n_workers=2)

In [12]:
future2

<Future at 0x7f7fb2d1a150 state=running>

In [16]:
future2.result()

[('a', 384993),
 ('b', 384058),
 ('c', 385432),
 ('d', 384097),
 ('e', 384582),
 ('f', 384730),
 ('g', 383708),
 ('h', 385596),
 ('i', 383945),
 ('j', 384994),
 ('k', 384845),
 ('l', 384452),
 ('m', 384036),
 ('n', 383996),
 ('o', 385117),
 ('p', 385352),
 ('q', 385935),
 ('r', 384729),
 ('s', 384508),
 ('t', 384910),
 ('u', 384548),
 ('v', 384364),
 ('w', 384810),
 ('x', 383658),
 ('y', 385116),
 ('z', 383489)]

In [19]:
!docker kill 34ec7ed17539 9a3f44ba8fad b75a0a1af63f 38ade40299e3

34ec7ed17539
9a3f44ba8fad
b75a0a1af63f
38ade40299e3


In [22]:
!docker kill 426f78306d63

426f78306d63


In [39]:
local_monitoring.print_zoo()


----------------- Zoo Masters -----------------
Master dc4bdb552ee5 :  MasterInfo(state='nothing')
Master fcca81584c9d :  MasterInfo(state='nothing')
Master 8d9938943886 :  MasterInfo(state='nothing')
Master e446fe4143ff :  MasterInfo(state='nothing')
Master d34f1fd4d9d7 :  MasterInfo(state='nothing')
Master d96ab6d00d3e :  MasterInfo(state='nothing')
Master 69d16b5229b0 :  MasterInfo(state='nothing')
Master 2a78850e1788 :  MasterInfo(state='nothing')
Master 30a2fe8cd20b :  MasterInfo(state='nothing')
Master a8416f608af5 :  MasterInfo(state='nothing')

----------------- Zoo Workers -----------------
Worker 986ea0638d8a :  WorkerInfo(state='in-task')
Worker c32afb8f5105 :  WorkerInfo(state='in-task')
Worker e6cabba3a6ac :  WorkerInfo(state='in-task')
Worker d534a0ff5782 :  WorkerInfo(state='in-task')
Worker dd167e9ea5b3 :  WorkerInfo(state='in-task')
Worker 0dfa4ac52fa8 :  WorkerInfo(state='in-task')
Worker f675fd3f64e9 :  WorkerInfo(state='in-task')
Worker 543b2431ad7a :  WorkerInfo(s

In [9]:
local_monitoring.print_hdfs('jobs')

In [37]:
local_cluster.scale(n_workers=10)

In [21]:
local_cluster.clear()

In [45]:
local_cluster.shutdown_cluster(cleanup=True)